In [3]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00


In [ ]:
import os
import torch
import pickle
import random
import string
import logging
import shutil
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from CiteHeteroSum.utils import *
from CiteHeteroSum.CiteHeteroSum_model.Loss import Triplet
from CiteHeteroSum.CiteHeteroSum_model.Training_Process import train_e2e
from CiteHeteroSum.CiteHeteroSum_model.Graph_Encoder import StepWiseGraphConvLayer, Contrast_Encoder, MLP, End2End_Encoder
from config import args

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [5]:
# output config
model_save_root_path = f'data/checkpoint/'
best_r2, best_loss, best_s_loss = 0, 10000, 10000
history = {'loss': [], 'val_loss': []}

# seed and device
seed_everything(args['seed'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## CONFIG

In [6]:
scenario = "1"
valid_scenarios = ["1", "2", "3"]

if scenario not in valid_scenarios:
    raise ValueError(f"Invalid scenario: {scenario}. Must be one of {valid_scenarios}")

In [7]:
import json
with open("./config.json", "r") as f:
    config_data = json.load(f)

train_file = config_data['Train'][f"scenario{scenario}"]["output"]
model_version = "".join(f"scenario{scenario}")


In [8]:
with open(train_file, "rb") as f:
    trainGraphs = pickle.load(f)

# Train model

In [9]:
import logging
logging.basicConfig(filename='app.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [10]:
c_graph_encoder = StepWiseGraphConvLayer(in_dim=768, out_dim=args['hidden'], hid_dim=args['hidden'],
                                         dropout_p=args['dropout'], act=nn.LeakyReLU(), nheads=args['heads'], iter=1).to(device)

contrast_filter = Contrast_Encoder(c_graph_encoder, args['hidden'], dropout_p=args['dropout']).to(device)
summarization_encoder = End2End_Encoder(768, args['hidden'], args['dropout']).to(device)

optimizer = torch.optim.Adam([ {'params': summarization_encoder.parameters()},
                            {'params': contrast_filter.parameters()}], lr=args['lr'], weight_decay=1e-5)

loss_method = Triplet(margin=args['triplet_margin'], topk=args['triplet_topk'])

In [11]:
eval_model_save_path = ""
c_patient = args["c_patient"]

for i in range(args['epochs']):
    print("Epoch {}".format(i))
    random.shuffle(trainGraphs)

    if c_patient < 0:
        for p in contrast_filter.parameters():
            p.requires_grad = False
        print("Stop Training Contrast")

    model = [contrast_filter, summarization_encoder]
    loss, rouge2_score = train_e2e(trainGraphs, model, optimizer, loss_method, args["summary_max_word_num"], edu_num=0, kappa=args['kappa'])
    history['loss'].append(loss)
    print("At Epoch {}, Train Loss: {}, R2 score: {}".format(i, loss, rouge2_score))
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    history['val_loss'].append(loss)

    rouge2_score = round(rouge2_score, 4)

    eval_model_save_path = os.path.join(model_save_root_path, "e_{}_{}.mdl".format(i, model_version))
    torch.save(summarization_encoder.state_dict(), eval_model_save_path)

    eval_model_save_path = os.path.join(model_save_root_path, "c_{}_{}.mdl".format(i, model_version))
    torch.save(contrast_filter.state_dict(), eval_model_save_path)

    if loss < best_loss and c_patient >= 0:
        best_loss = loss
        c_patient = 30
    else:
        c_patient -= 1

Epoch 0
Batch 0, Loss: 11.46833324432373
Batch 0, C-Loss: 8.017688751220703
Batch 0, S-Loss: 3.4506444931030273
Batch 20, Loss: 9.069693451835995
Batch 20, C-Loss: 7.257090613955543
Batch 20, S-Loss: 1.8126028577486675
At Epoch 0, Train Loss: 8.747398316860199, R2 score: 0.20466479782683225
Epoch 1
Batch 0, Loss: 9.501724243164062
Batch 0, C-Loss: 8.252777099609375
Batch 0, S-Loss: 1.2489466667175293
Batch 20, Loss: 7.892818110329764
Batch 20, C-Loss: 6.144753955659413
Batch 20, S-Loss: 1.7480641035806566
At Epoch 1, Train Loss: 7.299234992265701, R2 score: 0.22264264610898485
Epoch 2
Batch 0, Loss: 6.633899211883545
Batch 0, C-Loss: 4.8812255859375
Batch 0, S-Loss: 1.7526737451553345
Batch 20, Loss: 6.922525269644601
Batch 20, C-Loss: 5.379061744326637
Batch 20, S-Loss: 1.5434635764076596
At Epoch 2, Train Loss: 6.760447859764099, R2 score: 0.23165082742377413
Epoch 3
Batch 0, Loss: 5.393437385559082
Batch 0, C-Loss: 4.025966644287109
Batch 0, S-Loss: 1.3674707412719727
Batch 20, Loss